In [1]:
import numpy as np
import os
import glob

# Step 1: Define the root folder containing wavelet results
root_folder = 'wavelet_results_1'  # Root folder containing A_extended and E_extended

# Step 2: List of sub-band files to process (in order for MAV ratio calculations)
subbands = ['D1.txt', 'D2.txt', 'D3.txt', 'D4.txt', 'D5.txt', 'A5.txt']

# Step 3: Process both A_extended and E_extended folders
for data_type in ['A_extended', 'E_extended']:
    label = 1 if data_type == 'A_extended' else 0  # Assign label based on folder type
    data_folder = os.path.join(root_folder, data_type)

    # Step 4: Loop through each EEG file's folder
    folders = glob.glob(os.path.join(data_folder, '*'))

    for folder in folders:
        filename = os.path.basename(folder)  # Extract original filename (e.g., Z001)

        previous_mean = None  # Store the MAV of the previous sub-band for ratio calculations

        # Step 5: Create a folder to store results for the current EEG file
        results_folder = os.path.join(folder, 'calculated_metrics')
        os.makedirs(results_folder, exist_ok=True)  # Create folder if it doesn't exist

        # Step 6: Process each sub-band
        for subband in subbands:
            subband_path = os.path.join(folder, subband)

            if os.path.exists(subband_path):
                # Step 7: Load coefficients for the current sub-band
                coeffs = np.loadtxt(subband_path)

                # Step 8: Calculate metrics
                mav = np.mean(np.abs(coeffs))  # Mean of absolute values (MAV)
                avg_power = np.mean(coeffs ** 2)  # Average power
                std_dev = np.std(coeffs)  # Standard deviation

                # Step 9: Calculate ratio of MAVs between adjacent sub-bands
                ratio = mav / previous_mean if previous_mean is not None else np.nan
                previous_mean = mav  # Update previous_mean for the next sub-band

                # Step 10: Create a subfolder for the current sub-band (e.g., D1)
                subband_folder = os.path.join(results_folder, subband.replace('.txt', ''))
                os.makedirs(subband_folder, exist_ok=True)  # Create subfolder if not present

                # Step 11: Save metrics to a text file inside the sub-band folder
                metrics_file = os.path.join(subband_folder, 'metrics.txt')
                with open(metrics_file, 'w') as f:
                    f.write(f"Label: {label}\n")  # Save the label
                    f.write(f"MAV: {mav}\n")
                    f.write(f"Average Power: {avg_power}\n")
                    f.write(f"Standard Deviation: {std_dev}\n")
                    f.write(f"MAV Ratio: {ratio}\n")
            else:
                print(f"Sub-band file {subband} not found in {folder}.")
